In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [7]:
!python3 drive/Colab/mnist_cnn.py

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
2018-03-07 05:06:01.399716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-03-07 05:06:01.400264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1208] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.11GiB
2018

60000/60000 [==============================] - 13s 212us/step - loss: 0.2721 - acc: 0.9161 - val_loss: 0.0564 - val_acc: 0.9821
Epoch 2/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0871 - acc: 0.9743 - val_loss: 0.0433 - val_acc: 0.9859
Epoch 3/12
57600/60000 [===========================>..] - ETA: 0s - loss: 0.0633 - acc: 0.9816

60000/60000 [==============================] - 11s 181us/step - loss: 0.0634 - acc: 0.9815 - val_loss: 0.0359 - val_acc: 0.9880
Epoch 4/12
60000/60000 [==============================] - 11s 180us/step - loss: 0.0540 - acc: 0.9841 - val_loss: 0.0302 - val_acc: 0.9903
Epoch 5/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0453 - acc: 0.9861 - val_loss: 0.0353 - val_acc: 0.9885
Epoch 6/12
 4352/60000 [=>............................] - ETA: 9s - loss: 0.0353 - acc: 0.9901

60000/60000 [==============================] - 11s 181us/step - loss: 0.0406 - acc: 0.9874 - val_loss: 0.0291 - val_acc: 0.9911
Epoch 7/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0360 - acc: 0.9890 - val_loss: 0.0278 - val_acc: 0.9915
Epoch 8/12
55680/60000 [==========================>...] - ETA: 0s - loss: 0.0343 - acc: 0.9897

60000/60000 [==============================] - 11s 180us/step - loss: 0.0341 - acc: 0.9898 - val_loss: 0.0270 - val_acc: 0.9916
Epoch 9/12
60000/60000 [==============================] - 11s 181us/step - loss: 0.0314 - acc: 0.9905 - val_loss: 0.0259 - val_acc: 0.9923
Epoch 10/12
60000/60000 [==============================] - 11s 180us/step - loss: 0.0282 - acc: 0.9912 - val_loss: 0.0269 - val_acc: 0.9913
Epoch 11/12
 4608/60000 [=>............................] - ETA: 9s - loss: 0.0208 - acc: 0.9941

60000/60000 [==============================] - 11s 181us/step - loss: 0.0279 - acc: 0.9914 - val_loss: 0.0264 - val_acc: 0.9920
Epoch 12/12
60000/60000 [==============================] - 11s 183us/step - loss: 0.0243 - acc: 0.9922 - val_loss: 0.0300 - val_acc: 0.9908
Test loss: 0.030026931226026864
Test accuracy: 0.9908
